In [1]:
pip install pyinstaller


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- ----------------------

In [2]:
pip install qrcode[pil] pillow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pywin32

In [4]:
pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -----------------

In [5]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tkcalendar

In [29]:
import qrcode
from PIL import Image, ImageDraw, ImageFont, ImageTk, ImageWin  # Import ImageWin for printing
import tkinter as tk
from tkinter import filedialog, messagebox
import os
import cv2  # OpenCV for capturing photos from the webcam
import openpyxl  # For Excel functionality
from tkcalendar import Calendar  # For Date picker widget
import datetime  # For today's date
import win32print
import win32ui

# Function to generate a unique patient ID
def generate_patient_id():
    prefix = 'GKNMH'
    excel_file = r"C:\Users\ward\Downloads\id_cards\patient_registration.xlsx"  # Excel file holds to hold patient historical data
    
    # Load the existing data from the Excel file
    if os.path.exists(excel_file):
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
        existing_ids = [row[0].value for row in sheet.iter_rows(min_row=2, max_col=1)]  # Get patient IDs from column 1
    else:
        existing_ids = []

    # Start with a patient id number of 1000
    patient_id_number = 1000
    while f"{prefix}{patient_id_number}" in existing_ids:
        patient_id_number += 1

    patient_id = f"{prefix}{patient_id_number}"
    return patient_id

# Function to generate a QR code
def generate_qr_code(data, qr_filename):
    qr = qrcode.QRCode(version=1, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white")
    qr_img.save(qr_filename)

# Function to create a patient ID card with heading and proper alignment
def create_patient_id_card(patient_info, qr_filename, photo_path, output_filename):
    font = ImageFont.load_default()

    # Create a blank white card
    card_width, card_height = 600, 400
    card = Image.new("RGB", (card_width, card_height), "white")
    draw = ImageDraw.Draw(card)

    # Add logo and heading
    logo_path = r"C:\Users\ward\Downloads\id_cards\LOGO\Add a heading.png"  # Use a valid logo image file
    logo = Image.open(logo_path)
    logo_width = card_width  # Fill the header width
    logo_height = 80  # Height of the logo in the header
    logo = logo.resize((logo_width, logo_height))
    card.paste(logo, (0, 0))
    
    # Add a heading (Patient Id card) below the logo
    title_font = ImageFont.load_default()
    bbox = draw.textbbox((0, 0), "Patient Id card", font=title_font)
    text_width = bbox[2] - bbox[0]  # Width of the text
    heading_x = (card_width - text_width) // 2
    draw.text((heading_x, 90), "Patient Id card", font=title_font, fill="blue")

    # Add patient details on the left side
    detail_start_x, detail_start_y = 20, 120

    # Add patient info with proper alignment
    draw.text((detail_start_x, detail_start_y), f"Patient Name: {patient_info['name']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 40), f"Patient ID: {patient_info['id']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 80), f"Date of Birth: {patient_info['dob']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 120), f"Care Of: {patient_info['care_of']}", font=font, fill="black")

    # Add the QR code image to the card
    qr_img = Image.open(qr_filename)
    qr_img = qr_img.resize((100, 100))  # Resize the QR code to fit
    card.paste(qr_img, (card_width - 120, 280))  # Position the QR code on the card

    # Add the patient's photo to the card
    if os.path.exists(photo_path):
        photo = Image.open(photo_path)
        photo = photo.resize((80, 80))  # Resize the photo
        card.paste(photo, (card_width - 120, 180))  # Position the photo on the card

    # Add Registration Date at the bottom
    registration_date = patient_info["registration_date"]
    draw.text((detail_start_x, card_height - 40), f"Registration Date: {registration_date}", font=font, fill="black")

    # Save the card image
    card.save(output_filename)  # Save the ID card image

    print(f"ID card saved as {output_filename}")

# Function to write patient data to an Excel sheet
def write_to_excel(patient_info):
    excel_file = r"C:\Users\ward\Downloads\patient_registration.xlsx"  # Excel file holds to hold patient historical data
    
    if os.path.exists(excel_file):
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
    else:
        wb = openpyxl.Workbook()
        sheet = wb.active
        # Create the header
        sheet.append(["Patient ID", "Name", "Date of Birth", "Care Of", "Photo Path", "QR Code PWARDSath", "Registration Date"])

    # Write new patient data
    sheet.append([patient_info["id"], patient_info["name"], patient_info["dob"], patient_info["care_of"], patient_info["photo_path"], patient_info["qr_filename"], patient_info["registration_date"]])

    # Save the workbook
    wb.save(excel_file)
    print(f"Patient data saved to {excel_file}")

# Function to handle form submission
def submit_form():
    name = name_entry.get()
    patient_id = generate_patient_id()  # Automatically generate a unique patient ID
    dob = dob_entry.get()  # Get the date entered manually or selected from the calendar
    care_of = care_of_entry.get()
    photo_path = photo_path_var.get()

    if not name or not dob or not care_of or not photo_path:
        messagebox.showerror("Error", "Please fill in all fields and select a photo.")
        return

    folder_path = r"C:\Users\ward\Downloads\id_cards"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    qr_filename = "patient_qr.png"
    id_card_filename = os.path.join(folder_path, f"{patient_id}_id_card.png")
    qr_data = f"Name: {name}, ID: {patient_id}, DOB: {dob}, Care Of: {care_of}"
    generate_qr_code(qr_data, qr_filename)
    
    # Get today's date for registration date
    registration_date = datetime.datetime.today().strftime('%d-%m-%y')

    # Create the ID card
    create_patient_id_card(
        {"name": name, "id": patient_id, "dob": dob, "care_of": care_of, "registration_date": registration_date},
        qr_filename,
        photo_path,
        id_card_filename,
    )

    os.remove(qr_filename)  # Clean up temporary QR code image file

    messagebox.showinfo("Success", f"ID card saved as {id_card_filename}")

    # Log the patient info to the Excel file
    patient_info = {
        "name": name,
        "id": patient_id,
        "dob": dob,
        "care_of": care_of,
        "photo_path": photo_path,
        "qr_filename": qr_filename,
        "registration_date": registration_date
    }
    write_to_excel(patient_info)

    # Enable print button after successful generation
    print_button.config(state="normal")

    # Reset the form fields after successful submission
    name_entry.delete(0, tk.END)
    dob_entry.delete(0, tk.END)  # Clear the date entry field
    care_of_entry.delete(0, tk.END)
    photo_path_var.set("")  # Clear the photo path
    print_button.config(state="normal")  # Ensure it's enabled
    preview_id_card(id_card_filename)  # Show preview of the generated ID card

# Function to preview the ID card
def preview_id_card(id_card_filename):
    preview_image = Image.open(id_card_filename)  # Open the saved ID card image
    preview_image = preview_image.resize((300, 200))  # Resize the preview image to fit in the Tkinter window

    # Convert the image to a Tkinter-compatible format
    preview_photo = ImageTk.PhotoImage(preview_image)

    # Create a label to display the image
    preview_label = tk.Label(app, image=preview_photo)
    preview_label.image = preview_photo  # Keep a reference to the image
    preview_label.grid(row=7, column=0, columnspan=3, pady=10)

# Function to handle print action
def print_id_card(id_card_filename):
    # Load the image file
    image = Image.open(id_card_filename)
    
    # Set up the printer
    printer_name = win32print.GetDefaultPrinter()  # Get the default printer
    hprinter = win32print.OpenPrinter(printer_name)
    printer_info = win32print.GetPrinter(hprinter, 2)  # Retrieve printer info (needed to set page size)

    # Create a printer device context
    hdc = win32ui.CreateDC()
    hdc.CreatePrinterDC(printer_name)
    hdc.StartDoc(id_card_filename)  # Start a new document
    hdc.StartPage()  # Start a new page
    
    # Convert the image to a format suitable for printing
    img = ImageWin.Dib(image)  # Convert the PIL image to a format suitable for printing
    img.draw(hdc.GetHandleOutput(), (0, 0))  # Draw the image on the page

    # End the page and document
    hdc.EndPage()
    hdc.EndDoc()

    # Release the printer device context
    hdc.DeleteDC()

    # Show a message box indicating that the document was sent to the printer
    messagebox.showinfo("Print", f"ID card has been sent to {printer_name}.")

# Function to reset the form after printing or generating a new card
def reset_form():
    name_entry.delete(0, tk.END)
    dob_entry.delete(0, tk.END)
    care_of_entry.delete(0, tk.END)
    photo_path_var.set("")  # Clear the photo path
    print_button.config(state="disabled")  # Disable print button after printing

# Function to browse for a photo
def browse_photo():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    photo_path_var.set(file_path)

# Function to capture photo using the webcam
def capture_photo():
    cap = cv2.VideoCapture(0)  # Open webcam

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow("Press 'c' to capture or 'q' to quit", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):  # Capture photo on 'c' key press
            photo_filename = "captured_photo.png"
            cv2.imwrite(photo_filename, frame)
            photo_path_var.set(photo_filename)  # Update the photo path in the form
            break
        elif key == ord('q'):  # Quit on 'q' key press
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to show calendar and allow date selection
def show_calendar():
    dob_calendar.grid(row=1, column=1, padx=10, pady=5)
    dob_calendar.lift()  # Bring the calendar to the front

# Function to handle date selection from the calendar
def select_date():
    selected_date = dob_calendar.get_date()
    dob_entry.delete(0, tk.END)  # Clear the Entry widget
    dob_entry.insert(0, selected_date)  # Insert the selected date into the Entry widget
    dob_calendar.grid_remove()  # Hide the calendar after selection

# Create the GUI form
app = tk.Tk()
app.title("LyNx Digital Patient ID Creator")

# Name field
tk.Label(app, text="Patient Name:").grid(row=0, column=0, sticky="w", padx=10, pady=5)
name_entry = tk.Entry(app, width=30)
name_entry.grid(row=0, column=1, padx=10, pady=5)

# Date of Birth field (using Entry widget for manual input)
tk.Label(app, text="Date of Birth (YYYY-MM-DD):").grid(row=1, column=0, sticky="w", padx=10, pady=5)
dob_entry = tk.Entry(app, width=30)
dob_entry.grid(row=1, column=1, padx=10, pady=5)

# Button to show calendar
calendar_button = tk.Button(app, text="Pick Date", command=show_calendar)
calendar_button.grid(row=1, column=2, padx=10, pady=5)

# Care Of field
tk.Label(app, text="Care Of (Guardian/Relative):").grid(row=2, column=0, sticky="w", padx=10, pady=5)
care_of_entry = tk.Entry(app, width=30)
care_of_entry.grid(row=2, column=1, padx=10, pady=5)

# Photo field
tk.Label(app, text="Patient Photo:").grid(row=3, column=0, sticky="w", padx=10, pady=5)
photo_path_var = tk.StringVar()
photo_entry = tk.Entry(app, textvariable=photo_path_var, width=30, state="readonly")
photo_entry.grid(row=3, column=1, padx=10, pady=5)
browse_button = tk.Button(app, text="Browse", command=browse_photo)
browse_button.grid(row=4, column=2, padx=10, pady=5)

# Camera button
camera_button = tk.Button(app, text="Capture Photo", command=capture_photo)
camera_button.grid(row=3, column=2, padx=15, pady=10)

# Submit button
submit_button = tk.Button(app, text="Generate ID Card", command=submit_form)
submit_button.grid(row=5, column=0, columnspan=3, pady=10)

# Print button (disabled initially)
print_button = tk.Button(app, text="Print ID Card", state="disabled", command=lambda: print_id_card(f"{patient_id}_id_card.png"))
print_button.grid(row=6, column=0, columnspan=3, pady=10)

# Add the calendar widget but hide it initially
dob_calendar = Calendar(app, date_pattern="dd-mm-yyyy")
dob_calendar.grid(row=1, column=1, padx=10, pady=5)
dob_calendar.grid_remove()  # Hide the calendar by default

# Bind the date selection
dob_calendar.bind("<<CalendarSelected>>", lambda e: select_date())

app.mainloop()

ID card saved as C:\Users\ward\Downloads\id_cards\GKNMH1000_id_card.png
Patient data saved to C:\Users\ward\Downloads\patient_registration.xlsx


In [1]:
import qrcode
from PIL import Image, ImageDraw, ImageFont, ImageTk, ImageWin  # Import ImageWin for printing
import tkinter as tk
from tkinter import filedialog, messagebox
import os
import cv2  # OpenCV for capturing photos from the webcam
import openpyxl  # For Excel functionality
from tkcalendar import Calendar  # For Date picker widget
import datetime  # For today's date
import win32print
import win32ui

# Function to generate a unique patient ID
def generate_patient_id():
    prefix = 'GKNMH'
    excel_file = r"C:\Users\ward\Downloads\id_cards\patient_registration.xlsx"  # Excel file holds to hold patient historical data
    
    # Load the existing data from the Excel file
    if os.path.exists(excel_file):
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
        existing_ids = [row[0].value for row in sheet.iter_rows(min_row=2, max_col=1)]  # Get patient IDs from column 1
    else:
        existing_ids = []

    # Start with a patient id number of 1000
    patient_id_number = 1000
    while f"{prefix}{patient_id_number}" in existing_ids:
        patient_id_number += 1

    patient_id = f"{prefix}{patient_id_number}"
    return patient_id

# Function to generate a QR code
def generate_qr_code(data, qr_filename):
    qr = qrcode.QRCode(version=1, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white")
    qr_img.save(qr_filename)

# Function to create a patient ID card with heading and proper alignment
def create_patient_id_card(patient_info, qr_filename, photo_path, output_filename):
    font = ImageFont.load_default()

    # Create a blank white card
    card_width, card_height = 600, 400
    card = Image.new("RGB", (card_width, card_height), "white")
    draw = ImageDraw.Draw(card)

    # Add logo and heading
    logo_path = r"C:\Users\ward\Downloads\id_cards\LOGO\Add a heading.png"  # Use a valid logo image file
    logo = Image.open(logo_path)
    logo_width = card_width  # Fill the header width
    logo_height = 80  # Height of the logo in the header
    logo = logo.resize((logo_width, logo_height))
    card.paste(logo, (0, 0))
    
    # Add a heading (Patient Id card) below the logo
    title_font = ImageFont.load_default()
    bbox = draw.textbbox((0, 0), "Patient Id card", font=title_font)
    text_width = bbox[2] - bbox[0]  # Width of the text
    heading_x = (card_width - text_width) // 2
    draw.text((heading_x, 90), "Patient Id card", font=title_font, fill="blue")

    # Add patient details on the left side
    detail_start_x, detail_start_y = 20, 120

    # Add patient info with proper alignment
    draw.text((detail_start_x, detail_start_y), f"Patient Name: {patient_info['name']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 40), f"Patient ID: {patient_info['id']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 80), f"Date of Birth: {patient_info['dob']}", font=font, fill="black")
    draw.text((detail_start_x, detail_start_y + 120), f"Care Of: {patient_info['care_of']}", font=font, fill="black")

    # Add the QR code image to the card
    qr_img = Image.open(qr_filename)
    qr_img = qr_img.resize((100, 100))  # Resize the QR code to fit
    card.paste(qr_img, (card_width - 120, 280))  # Position the QR code on the card

    # Add the patient's photo to the card
    if os.path.exists(photo_path):
        photo = Image.open(photo_path)
        photo = photo.resize((80, 80))  # Resize the photo
        card.paste(photo, (card_width - 120, 180))  # Position the photo on the card

    # Add Registration Date at the bottom
    registration_date = patient_info["registration_date"]
    draw.text((detail_start_x, card_height - 40), f"Registration Date: {registration_date}", font=font, fill="black")

    # Save the card image
    card.save(output_filename)  # Save the ID card image

    print(f"ID card saved as {output_filename}")

# Function to write patient data to an Excel sheet
def write_to_excel(patient_info):
    excel_file = r"C:\Users\ward\Downloads\id_cards\patient_registration.xlsx"  # Excel file holds to hold patient historical data
    
    if os.path.exists(excel_file):
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
    else:
        wb = openpyxl.Workbook()
        sheet = wb.active
        # Create the header
        sheet.append(["Patient ID", "Name", "Date of Birth", "Care Of", "Photo Path", "QR Code", "Registration Date"])

    # Write new patient data
    sheet.append([patient_info["id"], patient_info["name"], patient_info["dob"], patient_info["care_of"], patient_info["photo_path"], patient_info["qr_filename"], patient_info["registration_date"]])

    # Save the workbook
    wb.save(excel_file)
    print(f"Patient data saved to {excel_file}")

# Function to handle form submission
def submit_form():
    name = name_entry.get()
    patient_id = generate_patient_id()  # Automatically generate a unique patient ID
    dob = dob_entry.get()  # Get the date entered manually or selected from the calendar
    care_of = care_of_entry.get()
    photo_path = photo_path_var.get()

    if not name or not dob or not care_of or not photo_path:
        messagebox.showerror("Error", "Please fill in all fields and select a photo.")
        return

    folder_path = r"C:\Users\ward\Downloads\id_cards"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    qr_filename = os.path.join(folder_path, "patient_qr.png")
    id_card_filename = os.path.join(folder_path, f"{patient_id}_id_card.png")
    qr_data = f"Name: {name}, ID: {patient_id}, DOB: {dob}, Care Of: {care_of}"
    generate_qr_code(qr_data, qr_filename)
    
    # Get today's date for registration date
    registration_date = datetime.datetime.today().strftime('%d-%m-%y')

    # Create the ID card
    create_patient_id_card(
        {"name": name, "id": patient_id, "dob": dob, "care_of": care_of, "registration_date": registration_date},
        qr_filename,
        photo_path,
        id_card_filename,
    )

    os.remove(qr_filename)  # Clean up temporary QR code image file

    messagebox.showinfo("Success", f"ID card saved as {id_card_filename}")

    # Log the patient info to the Excel file
    patient_info = {
        "name": name,
        "id": patient_id,
        "dob": dob,
        "care_of": care_of,
        "photo_path": photo_path,
        "qr_filename": qr_filename,
        "registration_date": registration_date
    }
    write_to_excel(patient_info)

    # Enable print button after successful generation
    print_button.config(state="normal")

    # Reset the form fields after successful submission
    name_entry.delete(0, tk.END)
    dob_entry.delete(0, tk.END)  # Clear the date entry field
    care_of_entry.delete(0, tk.END)
    photo_path_var.set("")  # Clear the photo path
    print_button.config(state="normal")  # Ensure it's enabled
    preview_id_card(id_card_filename)  # Show preview of the generated ID card

# Function to preview the ID card
def preview_id_card(id_card_filename):
    preview_image = Image.open(id_card_filename)  # Open the saved ID card image
    preview_image = preview_image.resize((300, 200))  # Resize the preview image to fit in the Tkinter window

    # Convert the image to a Tkinter-compatible format
    preview_photo = ImageTk.PhotoImage(preview_image)

    # Create a label to display the image
    preview_label = tk.Label(app, image=preview_photo)
    preview_label.image = preview_photo  # Keep a reference to the image
    preview_label.grid(row=7, column=0, columnspan=3, pady=10)

# Function to handle print action
def print_id_card(id_card_filename):
    # Load the image file
    image = Image.open(id_card_filename)
    
    # Set up the printer
    printer_name = win32print.GetDefaultPrinter()  # Get the default printer
    hprinter = win32print.OpenPrinter(printer_name)
    printer_info = win32print.GetPrinter(hprinter, 2)  # Retrieve printer info (needed to set page size)

    # Create a printer device context
    hdc = win32ui.CreateDC()
    hdc.CreatePrinterDC(printer_name)
    hdc.StartDoc(id_card_filename)  # Start a new document
    hdc.StartPage()  # Start a new page
    
    # Convert the image to a format suitable for printing
    img = ImageWin.Dib(image)  # Convert the PIL image to a format suitable for printing
    img.draw(hdc.GetHandleOutput(), (0, 0))  # Draw the image on the page

    # End the page and document
    hdc.EndPage()
    hdc.EndDoc()

    # Release the printer device context
    hdc.DeleteDC()

    # Show a message box indicating that the document was sent to the printer
    messagebox.showinfo("Print", f"ID card has been sent to {printer_name}.")

# Function to reset the form after printing or generating a new card
def reset_form():
    name_entry.delete(0, tk.END)
    dob_entry.delete(0, tk.END)
    care_of_entry.delete(0, tk.END)
    photo_path_var.set("")  # Clear the photo path
    print_button.config(state="disabled")  # Disable print button after printing

# Function to browse for a photo
def browse_photo():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    photo_path_var.set(file_path)

# Function to capture photo using the webcam
def capture_photo():
    cap = cv2.VideoCapture(0)  # Open webcam

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow("Press 'c' to capture or 'q' to quit", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):  # Capture photo on 'c' key press
            photo_filename = "captured_photo.png"
            cv2.imwrite(photo_filename, frame)
            photo_path_var.set(photo_filename)  # Update the photo path in the form
            break
        elif key == ord('q'):  # Quit on 'q' key press
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to show calendar and allow date selection
def show_calendar():
    dob_calendar.grid(row=1, column=1, padx=10, pady=5)
    dob_calendar.lift()  # Bring the calendar to the front

# Function to handle date selection from the calendar
def select_date():
    selected_date = dob_calendar.get_date()
    dob_entry.delete(0, tk.END)  # Clear the Entry widget
    dob_entry.insert(0, selected_date)  # Insert the selected date into the Entry widget
    dob_calendar.grid_remove()  # Hide the calendar after selection

# Create the GUI form
app = tk.Tk()
app.title("LyNx Digital Patient ID Creator")

# Name field
tk.Label(app, text="Patient Name:").grid(row=0, column=0, sticky="w", padx=10, pady=5)
name_entry = tk.Entry(app, width=30)
name_entry.grid(row=0, column=1, padx=10, pady=5)

# Date of Birth field (using Entry widget for manual input)
tk.Label(app, text="Date of Birth (YYYY-MM-DD):").grid(row=1, column=0, sticky="w", padx=10, pady=5)
dob_entry = tk.Entry(app, width=30)
dob_entry.grid(row=1, column=1, padx=10, pady=5)

# Button to show calendar
calendar_button = tk.Button(app, text="Pick Date", command=show_calendar)
calendar_button.grid(row=1, column=2, padx=10, pady=5)

# Care Of field
tk.Label(app, text="Care Of (Guardian/Relative):").grid(row=2, column=0, sticky="w", padx=10, pady=5)
care_of_entry = tk.Entry(app, width=30)
care_of_entry.grid(row=2, column=1, padx=10, pady=5)

# Photo field
tk.Label(app, text="Patient Photo:").grid(row=3, column=0, sticky="w", padx=10, pady=5)
photo_path_var = tk.StringVar()
photo_entry = tk.Entry(app, textvariable=photo_path_var, width=30, state="readonly")
photo_entry.grid(row=3, column=1, padx=10, pady=5)
browse_button = tk.Button(app, text="Browse", command=browse_photo)
browse_button.grid(row=4, column=2, padx=10, pady=5)

# Camera button
camera_button = tk.Button(app, text="Capture Photo", command=capture_photo)
camera_button.grid(row=3, column=2, padx=15, pady=10)

# Submit button
submit_button = tk.Button(app, text="Generate ID Card", command=submit_form)
submit_button.grid(row=5, column=0, columnspan=3, pady=10)

# Print button (disabled initially)
print_button = tk.Button(app, text="Print ID Card", state="disabled", command=lambda: print_id_card(f"{patient_id}_id_card.png"))
print_button.grid(row=6, column=0, columnspan=3, pady=10)

# Add the calendar widget but hide it initially
dob_calendar = Calendar(app, date_pattern="dd-mm-yyyy")
dob_calendar.grid(row=1, column=1, padx=10, pady=5)
dob_calendar.grid_remove()  # Hide the calendar by default

# Bind the date selection
dob_calendar.bind("<<CalendarSelected>>", lambda e: select_date())

app.mainloop()


ID card saved as C:\Users\ward\Downloads\id_cards\GKNMH1002_id_card.png
Patient data saved to C:\Users\ward\Downloads\id_cards\patient_registration.xlsx
